In [ ]:
from ndv.controller._controller import HistogramController
from ndv.data import cells3d

viewer = HistogramController()
viewer.data = cells3d()

viewer.view().show()

In [ ]:
# Change the data
from numpy.random import normal

shape = viewer.data.shape
viewer.data = normal(30000, 10000, shape)